In [ ]:
#### Kontruksi dan dokumentasi

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

print("Dimensi Data (Rows, Columns):", df.shape)
print("Informasi Data type dan encoding")
print(df.info())
print(df.head())

In [ ]:
print("Deskripsi Statistik")
print(df.describe())

print("Deteksi mising value")
print(df.isnull().sum())

# Identifikasi outlier menggunkan IQR
for column in df.columns[:-1]:
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = df[(df[column] < lower_bound) & (df[column] > upper_bound)]
    print(f"\nOutlier pada kolom {column}:")
    print(outliers)

#### Data Cleaning

In [ ]:
# Pembersihan data
# 1. Pengisian nilai yang hilang (jika ada)
# Dalam dataset iris, tidak ada nilai yang hilang, jadi bagian ini bisa dilewati.
# Namun, jika ada, Anda bisa menggunakan:
# df['sepal length (cm)'].fillna(df['sepal length (cm)'].mean(), inplace=True)
# df['sepal width (cm)'].fillna(df['sepal width (cm)'].median(), inplace=True)
# df['petal length (cm)'].fillna(df['petal length (cm)'].mode()[0], inplace=True)

# 2. Menghapus baris dengan data yang salah (jika ada)
# Dalam dataset iris, tidak ada data yang salah secara jelas.
# Namun, jika ada, Anda bisa menggunakan:
# df.drop(df[df['sepal length (cm)'] < 0].index, inplace=True)

# 3. Mengoreksi nilai outlier
for column in df.columns[:-1]:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)
    print(f"\nOutlier pada kolom {column}:")

# Setelah pembersihan data
print("\nDeskripsi statistik setelah pembersihan:")
print(df.describe())

#### Mengkontruksi Data

In [ ]:
# 1. Normalisasi Data
# Normalisasi dilakukan menggunakan MinMaxScaler untuk mengubah rentang nilai fitur menjadi 0-1.
scaler = MinMaxScaler()
numerical_features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
df[numerical_features] = scaler.fit_transform(df[numerical_features])
print("Data Setelah normalisasi")
print(df.head())

# 2. Pemilihan fitur
# Menggunakan SelectKBest untuk memilih 2 fitur terbaik berdasarkan skor ANOVA F-value.
X = df.drop('target', axis=1)
y = df['target']
selector = SelectKBest(score_func=f_classif, k=2)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Fitur terpilih", selected_features)

# 3.Binning
# Binning dilakukan pada fitur 'petal length (cm)' untuk mengelompokkan data menjadi beberapa kategori.
df['petal_length_binned'] = pd.cut(df['petal length (cm)'], bins=3, labels=['rendah', 'sedang', 'tinggi'])
print("Data setelah di binning")
print(df.head())

**Evaluasi Proses Transformasi Data**
Proses transformasi data meliputi normalisasi fitur-fitur numerik dan pembersihan nilai yang tidak sesuai. Berikut adalah evaluasinya:

- Normalisasi Data: Setiap fitur numerik (panjang dan lebar sepal serta petal) telah dinormalisasi ke rentang [0, 1]. Hal ini sangat membantu dalam mengurangi skala yang berbeda antara fitur dan membuat algoritma pembelajaran mesin lebih stabil dan cepat untuk berkonvergensi.Distribusi Data: Berdasarkan kuartil (25%, 50%, dan 75%) serta standar deviasi, data terlihat tersebar dengan baik dan tidak ada anomali yang signifikan dalam distribusi setelah normalisasi.
- Penanganan Missing Value: Tidak ada missing value dalam data setelah proses pembersihan ini, yang menunjukkan bahwa data telah dipersiapkan dengan baik untuk analisis lebih lanjut.

#### Menentukan Label Data

In [ ]:
# Identifikasi data target dan lakukan pelabelan
# Dalam dataset iris, kolom 'target' sudah mewakili label kelas (0, 1, 2).
# Namun, jika diperlukan pelabelan yang lebih spesifik berdasarkan SOP,
# kita bisa melakukan mapping label sesuai dengan kebutuhan.

# Misalnya, jika SOP menentukan bahwa:
# - target 0: Setosa
# - target 1: Versicolor
# - target 2: Virginica
# Kita bisa membuat mapping label seperti ini:

label_mapping = {
    0: "Setosa",
    1: "Versicolor",
    2: "Virginica",
}

# Buat kolom baru 'target_label' yang berisi label teks
df['target'] = df['target'].map(label_mapping)

# Tampilkan data dengan label baru
print("label baru berdasarkan SOP")
print(df.head())

**Evaluasi Hasil Pelabelan**
Kolom target mengandung label yang sudah diproses dengan baik:

- Jumlah Kelas: Dataset terdiri dari tiga kelas (0, 1, dan 2), yang terdistribusi secara proporsional berdasarkan mean dan kuartil data.
- Proporsi Kelas: Nilai mean dari 1.0 menunjukkan bahwa kelas-kelas dalam target tersebar rata dan tidak ada bias besar ke salah satu kelas tertentu.
- Kesimpulan Evaluasi Data

#### Viasualisasi Data

In [ ]:
# Histogram untuk melihat distribusi data setelah pembersihan
# Adjust the subplot grid to accommodate all columns
num_cols = len(df.columns[:-2])
num_rows = (num_cols + 1) // 2  # Calculate rows needed
plt.figure(figsize=(12, 6 * num_rows)) # Adjust figure height

for i, column in enumerate(df.columns[:-2]):
    plt.subplot(num_rows, 2, i + 1)
    sns.histplot(df[column], kde=True)
    plt.title(f"Distribusi {column}")
plt.tight_layout()
plt.show()

# Scatter plot untuk melihat hubungan antara dua fitur terpilih
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df[selected_features[0]], y=df[selected_features[1]], hue=df['target'])
plt.title(f'Hubungan antara {selected_features[0]} dan {selected_features[1]}')
plt.show()

# Box plot untuk melihat distribusi data sebelum dan sesudah normalisasi
plt.figure(figsize=(12,6))
for i, column in enumerate(numerical_features):
    plt.subplot(2, 2, i + 1)
    sns.boxplot(data=df, y=column)
    plt.title(f"Distribusi {column}")
plt.tight_layout()
plt.show()

# Histogram untuk melihat distribusi data setelah binning
plt.figure(figsize=(12,6))
sns.histplot(data=df['petal_length_binned'], kde=True)
plt.title(f"Distribusi Petal length setelah binning")
plt.show()

#### Contoh Laporan

**Kualitas Data yang Dibersihkan:**

- Data bebas dari missing value.
- Setiap fitur numerik telah dinormalisasi dengan baik sehingga tidak ada dominasi skala antara fitur.
- Statistik deskriptif menunjukkan distribusi data yang cukup baik tanpa outlier yang signifikan.

**Transformasi Data:**
- Normalisasi telah dilakukan dengan benar untuk menjaga kestabilan algoritma machine learning.
- Tidak ada fitur yang hilang atau terdistorsi dalam proses transformasi ini.

**Hasil Pelabelan:**
- Pelabelan target terlihat sudah sesuai dengan asumsi tiga kelas yang seimbang.
- Evaluasi target menunjukkan bahwa data siap digunakan untuk model prediksi tanpa risiko bias besar.

**Rekomendasi Selanjutnya**
- Lakukan analisis multivariat untuk mengeksplorasi korelasi antara fitur-fitur yang ada.
- Dengan data yang sudah bersih dan terstruktur, langkah selanjutnya adalah membangun model machine learning dan mengevaluasi performanya.
- Gunakan teknik validasi seperti cross-validation untuk memastikan bahwa model tidak mengalami overfitting.
